# goal
\
demo the OEMC dataset ahead of analysis

- note that the way I format districts may not be the best thing. Currently, I:\
    (1) preserve the original field,\
    (2) add a numeric version that removes padding 0s and formats as int,\
    (3) add a non-numeric version that preserves fields with a potentially meaningful non-numeric pattern, ie) "CW3".\
  The idea was to use the numeric version because that matches up with the district numbering in the open data portal, but that might not be the right call.
- note also that the `kwfields.yml` is not an exhaustive groupings of the `init_type` and `fin_type`s in the data and could be expanded on
- I don't know what "EL CHECK" is

# setup

In [1]:
# dependencies
import yaml
import re
import numpy as np
import pandas as pd

In [2]:
# support methods
def readyaml(fname):
    with open(fname, 'r') as f:
        data = yaml.safe_load(f)
    return data


def group_timedelta(td):
    if pd.isna(td): return 'No dispatch reported'
    elif td < pd.Timedelta(0): return 'Dispatch before call'
    elif td < pd.Timedelta(minutes=5): return 'Dispatch under 5 minutes'
    elif td < pd.Timedelta(minutes=15): return 'Dispatch under 15 minutes'
    elif td < pd.Timedelta(minutes=30): return 'Dispatch under 30 minutes'
    elif td < pd.Timedelta(minutes=60): return 'Dispatch under 1 hour'
    elif td < pd.Timedelta(minutes=120): return 'Dispatch under 2 hours'
    elif td < pd.Timedelta(minutes=360): return 'Dispatch under 6 hours'
    elif td < pd.Timedelta(days=.5): return 'Dispatch under 12 hours'
    elif td < pd.Timedelta(days=1): return 'Dispatch under 24 hours'
    elif td < pd.Timedelta(days=2): return 'Dispatch under 48 hours'
    return 'Dispatch 48 hours or later'

In [3]:
# main
oemc = pd.read_parquet("../data/oemc-prepped.parquet")
kwrules = readyaml("../data/hand/keywords.yml")
assert oemc.shape[0] > 12000000
assert not oemc.fin_type.isna().any()

# prep for analysis

- I thought this stuff was done in the version in `Chi-MP-data-story`, but it looks like I actually added these fields later and the version in the public repo is only lightly processed, not prepped for analysis.

**NOTE:** I'm not sure if the numeric district thing is the right approach or if it needs some tweaking, open to feedback and should review this before/when utilizing

In [4]:
oemc.loc[oemc.event_type == 'gun', ['event_group', 'event_type', 'init_type', 'fin_type']
].fillna("NO INITIAL TYPE").value_counts().head(50)

event_group  event_type  init_type                    fin_type                
help         gun         PERSON WITH A GUN            PERSON WITH A GUN           102438
                         SHOTS FIRED                  SHOTS FIRED                  71365
                         SHOTS FIRED (OV)             SHOTS FIRED (OV)             12671
                         PERSON SHOT                  PERSON SHOT                   8165
                         DISTURBANCE                  PERSON WITH A GUN              215
                         SHOTS FIRED                  PERSON SHOT                    173
                         PERSON WITH A GUN            ROBBERY JO                     138
                                                      SHOTS FIRED                    124
                         NO INITIAL TYPE              SHOTS FIRED (OV)               120
                         SELLING NARCOTICS            PERSON WITH A GUN              114
                         SHOTS 

In [5]:
oemc[['event_group', 'event_type', 'init_type', 'fin_type']
].fillna("NO INITIAL TYPE").value_counts().head(50)

event_group  event_type            init_type                  fin_type                 
surveil      traffic               TRAFFIC STOP (OV)          TRAFFIC STOP (OV)            2612639
             patrol                Community Interaction      Community Interaction         782470
other        other                 DISTURBANCE                DISTURBANCE                   541972
help         noinjury_report       AUTO ACCIDENT PD           AUTO ACCIDENT PD              452511
                                   DOMESTIC DISTURBANCE       DOMESTIC DISTURBANCE          412691
surveil      general               MISSION                    MISSION                       335607
help         noinjury_report       ALARM BURGLAR              ALARM BURGLAR                 298438
other        other                 EL CHECK [OV]              EL CHECK [OV]                 279423
help         noinjury_report       CHECK WELL BEING           CHECK WELL BEING              248942
surveil      traffic 

In [6]:
oemc.init_type.isna().sum()

np.int64(11921)

In [7]:
oemc.loc[oemc.init_type.isna(), 'fin_type'].value_counts()

fin_type
TRAFFIC STOP (OV)        6429
SHOT SPOTTER             2956
STREET STOP               533
Community Interaction     526
DISTURBANCE               164
                         ... 
CHILD LEFT ALONE            1
ANIMAL ABUSE                1
CRIMINAL TRESPASS IP        1
PROSTITUTION                1
THEFT JO                    1
Name: count, Length: 102, dtype: int64

# preview data

In [8]:
qa = f"Q:\tHow many records are in the OEMC dispatch data?\nA:\t{oemc.shape[0]:,} records"
print(qa)

Q:	How many records are in the OEMC dispatch data?
A:	12,159,582 records


In [9]:
oemc.sample().T

,774110
event_no,1809913254
district,003
call_date,2018-04-09 20:27:00
disp_date,2018-04-09 20:35:00
on_date,NaT
clear_date,2018-04-09 20:44:00
close_date,2018-04-09 20:44:00
init_priority,2C
init_type,SUSPICIOUS AUTO WITH OCC
fin_type,SUSPICIOUS AUTO WITH OCC


# review of data

### all fields

In [10]:
oemc.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12159582 entries, 0 to 12159581
Data columns (total 24 columns):
 #   Column               Dtype          
---  ------               -----          
 0   event_no             object         
 1   district             object         
 2   call_date            datetime64[ns] 
 3   disp_date            datetime64[ns] 
 4   on_date              datetime64[ns] 
 5   clear_date           datetime64[ns] 
 6   close_date           datetime64[ns] 
 7   init_priority        object         
 8   init_type            object         
 9   fin_type             object         
 10  year_called          int32          
 11  date_of_call         object         
 12  time_to_dispatch     timedelta64[ns]
 13  ttd_group            object         
 14  dispatch_reported    bool           
 15  closure_reported     bool           
 16  no_initial_type      bool           
 17  numeric_district     float64        
 18  nonnumeric_district  object         
 19  ini

In [11]:
for col in oemc.columns:
    print(f"Column name:\t`{col}`")
    print(f"N non-missing:\t{oemc[col].notna().sum():,}")
    print(f"N unique:\t{len(oemc[col].unique()):,}")
    print()

Column name:	`event_no`
N non-missing:	12,159,582
N unique:	12,159,582

Column name:	`district`
N non-missing:	12,159,582
N unique:	55

Column name:	`call_date`
N non-missing:	12,159,582
N unique:	2,039,809

Column name:	`disp_date`
N non-missing:	11,112,631
N unique:	2,023,674

Column name:	`on_date`
N non-missing:	8,518,965
N unique:	1,958,396

Column name:	`clear_date`
N non-missing:	11,103,705
N unique:	2,028,131

Column name:	`close_date`
N non-missing:	12,159,577
N unique:	2,043,149

Column name:	`init_priority`
N non-missing:	12,159,582
N unique:	13

Column name:	`init_type`
N non-missing:	12,147,661
N unique:	335

Column name:	`fin_type`
N non-missing:	12,159,582
N unique:	282

Column name:	`year_called`
N non-missing:	12,159,582
N unique:	4

Column name:	`date_of_call`
N non-missing:	12,159,582
N unique:	1,461

Column name:	`time_to_dispatch`
N non-missing:	11,112,631
N unique:	860

Column name:	`ttd_group`
N non-missing:	12,159,582
N unique:	12

Column name:	`dispatch_reporte

### original

- not inclusive of every single original field
- might have light processing or formatting applied

In [12]:
oemc.call_date.describe()

count                         12159582
mean     2019-12-17 23:36:37.666096384
min                2018-01-01 00:00:00
25%                2018-12-20 22:14:00
50%                2019-11-20 09:53:00
75%                2020-12-30 16:09:00
max                2021-12-31 23:59:00
Name: call_date, dtype: object

In [13]:
oemc.district.value_counts().head(20)

district
011    767079
008    686940
006    674354
004    640078
007    632357
025    621355
002    601269
003    594035
012    580341
010    567871
015    537204
001    510360
018    502664
005    472350
009    471412
019    455477
016    375640
024    360724
014    348870
CW4    336551
Name: count, dtype: int64

In [14]:
oemc[['init_type', 'fin_type']].value_counts().head(10)

init_type              fin_type             
TRAFFIC STOP (OV)      TRAFFIC STOP (OV)        2612639
Community Interaction  Community Interaction     782470
DISTURBANCE            DISTURBANCE               541972
AUTO ACCIDENT PD       AUTO ACCIDENT PD          452511
DOMESTIC DISTURBANCE   DOMESTIC DISTURBANCE      412691
MISSION                MISSION                   335607
ALARM BURGLAR          ALARM BURGLAR             298438
EL CHECK [OV]          EL CHECK [OV]             279423
CHECK WELL BEING       CHECK WELL BEING          248942
PARKING VIOL. 1        PARKING VIOL. 1           247982
Name: count, dtype: int64

In [15]:
oemc[['init_priority', 'init_type']].value_counts().head(10)

init_priority  init_type            
3D             TRAFFIC STOP (OV)        2619243
4              Community Interaction     782915
3B             DISTURBANCE               525177
3C             AUTO ACCIDENT PD          440115
1A             DOMESTIC DISTURBANCE      416420
4              MISSION                   336072
2B             ALARM BURGLAR             298772
4              EL CHECK [OV]             280298
1A             CHECK WELL BEING          250898
3D             PARKING VIOL. 1           249208
Name: count, dtype: int64

### added for analysis

- might have light processing or formatting applied

In [16]:
oemc.event_group.value_counts()

event_group
surveil    6025500
help       3222865
other      2911217
Name: count, dtype: int64

In [18]:
oemc.loc[oemc.init_type.str.contains("GENERIC", na=False), ['init_type', 'fin_type']].value_counts()

init_type                  fin_type                 
GENERIC (CHANGE PRIORITY)  GENERIC (CHANGE PRIORITY)    14670
                           Community Interaction          587
                           TRAFFIC STOP (OV)              141
                           PARK CHECK [OV]                 60
                           PREMISE CHECK [OV]              58
                                                        ...  
                           BOMB THREAT                      1
                           BATTERY JO                       1
                           SCHOOL CROSSING                  1
                           SEARCH WARRANT                   1
                           LOCK OUT                         1
Name: count, Length: 111, dtype: int64

In [19]:
oemc.event_type.value_counts()

event_type
traffic                 3029128
other                   2723229
patrol                  2189020
noinjury_report         2115073
injury_report            851238
general                  677321
gun                      198430
shotspotter_reclass?     189363
shotspotter               97889
mp                        57879
hunchlab                  28731
licplate                   2281
Name: count, dtype: int64

In [20]:
oemc.year_called.value_counts().sort_index()

year_called
2018    3119334
2019    3293586
2020    2715865
2021    3030797
Name: count, dtype: int64

In [21]:
oemc.time_to_dispatch.describe()

count                     11112631
mean     0 days 00:05:30.539221539
std      0 days 00:17:54.836038658
min              -1 days +11:36:00
25%                0 days 00:00:00
50%                0 days 00:00:00
75%                0 days 00:03:00
max                4 days 11:52:00
Name: time_to_dispatch, dtype: object

In [22]:
oemc.ttd_group.value_counts()

ttd_group
Dispatch under 5 minutes      8814501
Dispatch under 15 minutes     1202478
No dispatch reported          1046951
Dispatch under 30 minutes      522303
Dispatch under 1 hour          369036
Dispatch under 2 hours         169454
Dispatch under 6 hours          32407
Dispatch under 12 hours          2232
Dispatch under 24 hours           119
Dispatch before call               81
Dispatch under 48 hours            13
Dispatch 48 hours or later          7
Name: count, dtype: int64

In [23]:
oemc.numeric_district.value_counts()

numeric_district
11.0     767079
8.0      686940
6.0      674354
4.0      640078
7.0      632357
25.0     621355
2.0      601269
3.0      594035
12.0     580341
10.0     567871
15.0     537204
1.0      510360
18.0     502664
5.0      472350
9.0      471412
19.0     455477
16.0     375640
24.0     360724
14.0     348870
17.0     334756
22.0     326913
20.0     277942
31.0      30891
21.0        136
377.0        56
13.0         46
177.0        35
61.0         33
477.0        25
41.0         11
70.0          3
23.0          1
Name: count, dtype: int64

# topics

